# Imports

In [1]:
import pandas as pd
import os
from functions.utilities import load_data, clean_data, filter_by_common_years, filter_by_user_year_range

from functions.task1_functions import quality_of_movies_by_country, prepare_data

from functions.task2_functions import total_votes_by_country, average_composite_score_by_country, weighted_average_composite_score_by_country, filter_countries_with_reference, get_countries_and_clean_orders, calculate_gdp_per_population, rename_and_add_rank, compute_hegemony

from functions.task3_functions import prepare_movies_directors, rank_directors, custom_ranking, rank_director_actors

# Loading and preparing the data

#### Data for TASK 1

In [2]:
data_dir = os.getenv('MOVIE_DATA_PATH')
if data_dir is None:
    data_dir = 'data_imdb'

basics = clean_data(load_data(os.path.join(data_dir, 'title.basics.tsv')))
akas = clean_data(load_data(os.path.join(data_dir, 'title.akas.tsv')))
ratings = clean_data(load_data(os.path.join(data_dir, 'title.ratings.tsv')))

Loading data from: C:\Users\konra\PycharmProjects\Imdb_Film_Ranking\data_imdb\title.basics.tsv ...


Loading data from: C:\Users\konra\PycharmProjects\Imdb_Film_Ranking\data_imdb\title.akas.tsv ...


Loading data from: C:\Users\konra\PycharmProjects\Imdb_Film_Ranking\data_imdb\title.ratings.tsv ...


### Data for TASK 2

In [3]:
# GDP and Population data from Worldbank
# Source: https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?end=2023&name_desc=false&skipRedirection=true&start=1960&view=chart

data_dir = os.getenv('GDP_POP_DATA_PATH')
if data_dir is None:
    data_dir = 'data_gdp_population'

gdp_df = clean_data(load_data(os.path.join(data_dir, 'API_NY.GDP.MKTP.CD_DS2_en_csv_v2_580250.csv'), header=2))
pop_df = clean_data(load_data(os.path.join(data_dir, 'API_SP.POP.TOTL_DS2_en_csv_v2_580248.csv'), header=2))

# Introducing dataset with codenames for countries 
# Source: https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv
country_codes_df = pd.read_csv(data_dir + '/country_codes_all.csv')

Loading data from: C:\Users\konra\PycharmProjects\Imdb_Film_Ranking\data_gdp_population\API_NY.GDP.MKTP.CD_DS2_en_csv_v2_580250.csv ...
Loading data from: C:\Users\konra\PycharmProjects\Imdb_Film_Ranking\data_gdp_population\API_SP.POP.TOTL_DS2_en_csv_v2_580248.csv ...


### Data for TASK 3

In [4]:
data_dir = os.getenv('MOVIE_DATA_PATH')
if data_dir is None:
    data_dir = 'data_imdb'

crew = clean_data(load_data(os.path.join(data_dir, 'title.crew.tsv')))
names = clean_data(load_data(os.path.join(data_dir, 'name.basics.tsv')))

Loading data from: data_imdb\title.crew.tsv ...


Loading data from: data_imdb\name.basics.tsv ...


In [5]:
basics, gdp_df = filter_by_common_years(basics, gdp_df)

start_year = os.getenv('START_YEAR')
end_year = os.getenv('END_YEAR')

# Convert start_year and end_year to integers if they exist
if start_year is not None:
    start_year = int(start_year)
if end_year is not None:
    end_year = int(end_year)
    
basics, gdp_df = filter_by_user_year_range(basics, gdp_df, start_year, end_year)

In [6]:
print("Sample data from basics:")
basics.head()

Sample data from basics:


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
919,tt0000929,short,Klebolin klebt alles,Klebolin klebt alles,0,1990.0,<NA>,<NA>,"Comedy,Short"
965,tt0000977,short,Mutterliebe,Mutterliebe,0,1990.0,<NA>,<NA>,Short
11634,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,0,2019.0,<NA>,<NA>,"Action,Crime"
13079,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021.0,<NA>,94,Documentary
15174,tt0015414,movie,La tierra de los toros,La tierra de los toros,0,2000.0,<NA>,60,<NA>


In [7]:
print("Sample data from akas:")
akas.head()

Sample data from akas:


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita,<NA>,<NA>,original,<NA>,1
1,tt0000001,2,Carmencita,DE,<NA>,<NA>,literal title,0
2,tt0000001,3,Carmencita,US,<NA>,imdbDisplay,<NA>,0
3,tt0000001,4,Carmencita - spanyol tánc,HU,<NA>,imdbDisplay,<NA>,0
4,tt0000001,5,Καρμενσίτα,GR,<NA>,imdbDisplay,<NA>,0


In [8]:
print("Sample data from ratings:")
ratings.head()

Sample data from ratings:


,tconst,averageRating,numVotes
0,tt0000001,5.7,2058
1,tt0000002,5.7,276
2,tt0000003,6.5,2022
3,tt0000004,5.4,179
4,tt0000005,6.2,2787


# Task 1 - Quality of movies by country

In [9]:
top_orders = [10, 20, 50, 100]

# Prepare the data, merge needed datasets and filter of only movies
prepared_df = prepare_data(basics, ratings, akas)
prepared_df.head()

# Analyze the quality of movies by country
country_counts, movies_df = quality_of_movies_by_country(prepared_df, top_orders)

# Display country appearances in top N sequences
country_counts

There are 5650 movies without an assigned country.


{10: {'GB': 2,
  'IS': 1,
  'ID': 1,
  'FI': 1,
  'DK': 1,
  'EE': 1,
  'AE': 1,
  'HK': 1,
  'NZ': 1},
 20: {'IN': 4,
  'GB': 3,
  'AE': 2,
  'ID': 2,
  'NZ': 2,
  'IS': 1,
  'FI': 1,
  'DK': 1,
  'HK': 1,
  'EE': 1,
  'EG': 1,
  'PH': 1},
 50: {'IN': 12,
  'GB': 5,
  'AE': 4,
  'NL': 4,
  'IL': 2,
  'IT': 2,
  'ID': 2,
  'EE': 2,
  'EG': 2,
  'NZ': 2,
  'PH': 2,
  'DK': 2,
  'ES': 2,
  'IS': 1,
  'FI': 1,
  'HK': 1,
  'CM': 1,
  'CZ': 1,
  'EC': 1,
  'FR': 1},
 100: {'IN': 24,
  'ID': 7,
  'AE': 7,
  'NL': 7,
  'GB': 6,
  'EG': 5,
  'IL': 4,
  'PH': 4,
  'IE': 4,
  'FR': 4,
  'DK': 3,
  'HK': 3,
  'ES': 3,
  'IT': 2,
  'NZ': 2,
  'EE': 2,
  'EC': 2,
  'FI': 1,
  'IS': 1,
  'CZ': 1,
  'CM': 1,
  'CO': 1,
  'KR': 1,
  'GR': 1,
  'AU': 1,
  'JP': 1,
  'SG': 1,
  'DE': 1}}

In [10]:
# Master movie dataframe for further analyses
print(movies_df.shape)
movies_df.head(20)

(473429, 22)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,...,ordering,title,region,language,types,attributes,isOriginalTitle,titleId_y,country,composite_score
489982,tt0111161,movie,The Shawshank Redemption,The Shawshank Redemption,0,1994.0,<NA>,142,Drama,9.3,...,1.0,The Shawshank Redemption,<NA>,<NA>,original,<NA>,1.0,tt0111161,IS,871760.31
1020895,tt0468569,movie,The Dark Knight,The Dark Knight,0,2008.0,<NA>,152,"Action,Crime,Drama",9.0,...,1.0,The Dark Knight,<NA>,<NA>,original,<NA>,1.0,tt0468569,ID,866137.50
1397692,tt1375666,movie,Inception,Inception,0,2010.0,<NA>,148,"Action,Adventure,Sci-Fi",8.8,...,1.0,Inception,<NA>,<NA>,original,<NA>,1.0,tt1375666,GB,769596.16
589770,tt0137523,movie,Fight Club,Fight Club,0,1999.0,<NA>,139,Drama,8.8,...,1.0,Fight Club,<NA>,<NA>,original,<NA>,1.0,tt0137523,FI,701755.36
480389,tt0109830,movie,Forrest Gump,Forrest Gump,0,1994.0,<NA>,142,"Drama,Romance",8.8,...,1.0,Forrest Gump,<NA>,<NA>,original,<NA>,1.0,tt0109830,DK,681472.06
488060,tt0110912,movie,Pulp Fiction,Pulp Fiction,0,1994.0,<NA>,154,"Crime,Drama",8.9,...,1.0,Pulp Fiction,<NA>,<NA>,original,<NA>,1.0,tt0110912,EE,670144.43
1067954,tt0816692,movie,Interstellar,Interstellar,0,2014.0,<NA>,169,"Adventure,Drama,Sci-Fi",8.7,...,1.0,Interstellar,<NA>,<NA>,original,<NA>,1.0,tt0816692,GB,635870.19
582780,tt0133093,movie,The Matrix,The Matrix,0,1999.0,<NA>,136,"Action,Sci-Fi",8.7,...,1.0,The Matrix,<NA>,<NA>,original,<NA>,1.0,tt0133093,AE,619520.79
152918,tt0068646,movie,The Godfather,The Godfather,0,1972.0,<NA>,175,"Crime,Drama",9.2,...,1.0,The Godfather,<NA>,<NA>,original,<NA>,1.0,tt0068646,HK,607480.04
555043,tt0120737,movie,The Lord of the Rings: The Fellowship of the Ring,The Lord of the Rings: The Fellowship of the Ring,0,2001.0,<NA>,178,"Action,Adventure,Drama",8.9,...,1.0,The Lord of the Rings: The Fellowship of the Ring,<NA>,<NA>,original,<NA>,1.0,tt0120737,NZ,605482.43


# Task 2 - "Cinematic Impact" Hegemony

In [11]:
# weak cinematic impact
votes_df = total_votes_by_country(movies_df)

# strong cinematic impact
avg_score_df = average_composite_score_by_country(movies_df)

# strong cinematic impact weighted
avg_wgt_score_df = weighted_average_composite_score_by_country(movies_df)

In [12]:
# Creating orders of countries according to number of votes and scores for each country
votes_df, excluded_countries = get_countries_and_clean_orders(
    votes_df, country_codes_df, 'country', 'alpha-2', ['name', 'number of votes'])

avg_score_df, _ = get_countries_and_clean_orders(
    avg_score_df, country_codes_df, 'country', 'alpha-2',['name', 'average composite score'])

avg_wgt_score_df, _ = get_countries_and_clean_orders(
    avg_wgt_score_df, country_codes_df, 'country', 'alpha-2', ['name', 'weighted average composite score'])

print(f"Country codes excluded from further analysis:\n{excluded_countries}")

Country codes excluded from further analysis:
['XWW', 'XWG', 'CSHH', 'XYU', 'DDDE', 'XEU', 'SUHH', 'YUCS', 'CSXX', 'XAS', 'XKV', 'AN', 'BUMM', 'VDVN', 'XKO', 'ZRCD']


In [13]:
votes_df

,name,number of votes
0,India,202617537.0
1,Australia,82645411.0
2,United Kingdom of Great Britain and Northern I...,82238946.0
3,Indonesia,57949944.0
4,"Netherlands, Kingdom of the",47069448.0
...,...,...
221,Cayman Islands,0.0
222,Montserrat,0.0
223,Réunion,0.0
224,Saint Vincent and the Grenadines,0.0


In [14]:
avg_score_df

,name,average composite score
0,Cameroon,13940.579286
1,Ireland,13253.603111
2,Israel,9044.869101
3,Indonesia,7576.234935
4,New Zealand,7107.316941
...,...,...
221,Cayman Islands,NaN
222,Montserrat,NaN
223,Réunion,NaN
224,Saint Vincent and the Grenadines,NaN


In [15]:
# Filtering and ordering Countries and their GDPs
gdp_df = filter_countries_with_reference(gdp_df, 'Country Code', country_codes_df, 'alpha-3', year='2023')
gdp_df

,Country Name,2023
251,United States,2.736094e+13
40,China,1.779478e+13
55,Germany,4.456081e+12
119,Japan,4.212945e+12
109,India,3.549919e+12
...,...,...
239,Tonga,NaN
254,"Venezuela, RB",NaN
255,British Virgin Islands,NaN
256,Virgin Islands (U.S.),NaN


In [16]:
# Filtering and ordering Countries and their Populations
pop_df = filter_countries_with_reference(pop_df, 'Country Code', country_codes_df, 'alpha-3', year='2023')
pop_df.head(10)

,Country Name,2023
109,India,1.428628e+09
40,China,1.410710e+09
251,United States,3.349149e+08
106,Indonesia,2.775341e+08
184,Pakistan,2.404857e+08
174,Nigeria,2.238046e+08
29,Brazil,2.164224e+08
20,Bangladesh,1.729543e+08
202,Russian Federation,1.438261e+08
154,Mexico,1.284556e+08


In [17]:
# Calculating and ordering Countries and their GDP/Population coefficient
gdp_pop_df = calculate_gdp_per_population(gdp_df, pop_df, '2023')
gdp_pop_df.head(10)

,index,gdp_per_population
70,Luxembourg,128259.402583
24,Ireland,103684.880802
19,Switzerland,99994.938020
30,Norway,87961.780614
29,Singapore,84734.255921
0,United States,81695.187071
103,Iceland,78811.058392
34,Denmark,67967.381869
92,"Macao SAR, China",66835.064334
12,Australia,64711.765603


In [18]:
# Renaming given columns for better cohesion and creating rankings for each relevant variable
votes_df = rename_and_add_rank(votes_df, ['country', 'number of votes'])
avg_score_df = rename_and_add_rank(avg_score_df, ['country', 'average score'])
avg_wgt_score_df = rename_and_add_rank(avg_wgt_score_df, ['country', 'weighted average score'])

gdp_df = rename_and_add_rank(gdp_df, ['country', 'gdp'])
pop_df = rename_and_add_rank(pop_df, ['country', 'population'])
gdp_pop_df = rename_and_add_rank(gdp_pop_df, ['country', 'gdp/population'])

In [19]:
# Calculating Hegemony Scores in various combinations
gdp_score_hegemony = compute_hegemony(gdp_df, avg_score_df, 'gdp', 'avg_score')
pop_votes_hegemony = compute_hegemony(pop_df, votes_df, 'pop', 'votes')
gdp_pop_wgt_score_hegemony = compute_hegemony(gdp_pop_df, avg_wgt_score_df, 'gdp_pop', 'wgt_score')


Gdp / Avg_score Hegemony Rankings:
1. Thailand (Hegemony Score: 0)
2. Albania (Hegemony Score: 0)
3. Slovenia (Hegemony Score: 1)
4. Sweden (Hegemony Score: 1)
5. Mali (Hegemony Score: 1)
6. Marshall Islands (Hegemony Score: 1)
7. Mongolia (Hegemony Score: 2)
8. Myanmar (Hegemony Score: 2)
9. Grenada (Hegemony Score: 3)
10. Bermuda (Hegemony Score: 3)
11. Sao Tome and Principe (Hegemony Score: 4)
12. Eritrea (Hegemony Score: 4)
13. Portugal (Hegemony Score: 4)
14. Azerbaijan (Hegemony Score: 5)
15. Antigua and Barbuda (Hegemony Score: 5)
16. Nepal (Hegemony Score: 5)
17. Cyprus (Hegemony Score: 6)
18. Norway (Hegemony Score: 6)
19. Belize (Hegemony Score: 7)
20. American Samoa (Hegemony Score: 7)
21. Belarus (Hegemony Score: 7)
22. Nicaragua (Hegemony Score: 8)
23. Romania (Hegemony Score: 10)
24. Guam (Hegemony Score: 10)
25. Armenia (Hegemony Score: 10)
26. Mauritania (Hegemony Score: 10)
27. Cabo Verde (Hegemony Score: 11)
28. Bulgaria (Hegemony Score: 11)
29. Indonesia (Hegemony S

# Task 3 - Focus on Directors

In [20]:
# Preparing data for directors analysis by merging movie_df with data from which inference about directors is made
movies_directors_df = prepare_movies_directors(crew, names, movies_df)
movies_directors_df.head(10)

,tconst,directors,writers,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,titleType,...,ordering,title,region,language,types,attributes,isOriginalTitle,titleId_y,country,composite_score
0,tt0111161,nm0001104,"nm0000175,nm0001104",nm0001104,Frank Darabont,1959,<NA>,"writer,producer,director","tt0120689,tt0111161,tt0884328,tt1520211",movie,...,1.0,The Shawshank Redemption,<NA>,<NA>,original,<NA>,1.0,tt0111161,IS,871760.31
1,tt0468569,nm0634240,"nm0634300,nm0634240,nm0275286,nm0004170",nm0634240,Christopher Nolan,1970,<NA>,"writer,producer,director","tt6723592,tt0816692,tt1375666,tt0482571",movie,...,1.0,The Dark Knight,<NA>,<NA>,original,<NA>,1.0,tt0468569,ID,866137.50
2,tt1375666,nm0634240,nm0634240,nm0634240,Christopher Nolan,1970,<NA>,"writer,producer,director","tt6723592,tt0816692,tt1375666,tt0482571",movie,...,1.0,Inception,<NA>,<NA>,original,<NA>,1.0,tt1375666,GB,769596.16
3,tt0137523,nm0000399,"nm0657333,nm0880243",nm0000399,David Fincher,1962,<NA>,"producer,director,writer","tt0114369,tt2267998,tt0443706,tt1285016",movie,...,1.0,Fight Club,<NA>,<NA>,original,<NA>,1.0,tt0137523,FI,701755.36
4,tt0109830,nm0000709,"nm0343165,nm0744839",nm0000709,Robert Zemeckis,1952,<NA>,"producer,writer,director","tt0088763,tt0109830,tt0118884,tt1907668",movie,...,1.0,Forrest Gump,<NA>,<NA>,original,<NA>,1.0,tt0109830,DK,681472.06
5,tt0110912,nm0000233,"nm0000233,nm0000812",nm0000233,Quentin Tarantino,1963,<NA>,"producer,writer,actor","tt0105236,tt0110912,tt7131622,tt0116367",movie,...,1.0,Pulp Fiction,<NA>,<NA>,original,<NA>,1.0,tt0110912,EE,670144.43
6,tt0816692,nm0634240,"nm0634300,nm0634240",nm0634240,Christopher Nolan,1970,<NA>,"writer,producer,director","tt6723592,tt0816692,tt1375666,tt0482571",movie,...,1.0,Interstellar,<NA>,<NA>,original,<NA>,1.0,tt0816692,GB,635870.19
8,tt0068646,nm0000338,"nm0701374,nm0000338",nm0000338,Francis Ford Coppola,1939,<NA>,"producer,director,writer","tt0078788,tt0071360,tt0068646,tt0071562",movie,...,1.0,The Godfather,<NA>,<NA>,original,<NA>,1.0,tt0068646,HK,607480.04
9,tt0120737,nm0001392,"nm0866058,nm0909638,nm0101991,nm0001392",nm0001392,Peter Jackson,1961,<NA>,"producer,director,writer","tt0120737,tt0092610,tt0167260,tt0360717",movie,...,1.0,The Lord of the Rings: The Fellowship of the Ring,<NA>,<NA>,original,<NA>,1.0,tt0120737,NZ,605482.43
10,tt0167260,nm0001392,"nm0866058,nm0909638,nm0101991,nm0001392",nm0001392,Peter Jackson,1961,<NA>,"producer,director,writer","tt0120737,tt0092610,tt0167260,tt0360717",movie,...,1.0,The Lord of the Rings: The Return of the King,<NA>,<NA>,original,<NA>,1.0,tt0167260,IN,596937.30


### Which director is the best according to our composite score? (this analysis' own metric)

In [21]:
# Using mean as aggregation function
rank_directors(movies_directors_df, 'primaryName', 'composite_score', 'mean')

,primaryName,aggregated_score,total_movies,rank
0,Christopher Nolan,407315.641667,13,1.0
1,Frank Darabont,354156.782500,4,2.0
2,John Lasseter,322166.810000,1,3.0
3,Quentin Tarantino,271842.562500,12,4.0
4,Lee Unkrich,268908.110000,1,5.0
...,...,...,...,...
167720,Ülo Tambek,NaN,1,NaN
167721,Ümit Volkan,NaN,1,NaN
167722,Ünsel Aybek,NaN,3,NaN
167723,Þorstein J. Vilhjálmsson,NaN,1,NaN


In [22]:
# Using sum as aggregation function
rank_directors(movies_directors_df, 'primaryName', 'composite_score', 'sum')    

,primaryName,aggregated_score,total_movies,rank
0,Christopher Nolan,4887787.70,13,1.0
1,Steven Spielberg,4371850.41,34,2.0
2,Quentin Tarantino,3262110.75,12,3.0
3,Martin Scorsese,3193269.69,36,4.0
4,David Fincher,2731575.10,12,5.0
...,...,...,...,...
167720,Þorsteinn Helgason,0.00,1,99830.0
167721,.K. Sharath,0.00,1,99830.0
167722,.K. Nanjunda,0.00,1,99830.0
167723,'Rayzor' Raymond Davis,0.00,1,99830.0


In [23]:
# Using thresholds for good and bad movies to enhance decent movies and punish flops. Looking for more predictable directors.
custom_ranking(movies_directors_df, 'primaryName', 'composite_score', good_threshold=7.0, bad_threshold=3.0)

,primaryName,custom_score,total_movies,rank
0,Christopher Nolan,63.525089,13,1.0
1,Frank Darabont,20.122436,4,2.0
2,Quentin Tarantino,18.051022,12,3.0
3,David Fincher,9.358073,12,4.0
4,John Lasseter,1.391147,1,5.0
...,...,...,...,...
167720,Kostas Karagiannis,-302.941768,102,167721.0
167721,Ülkü Erakalin,-302.983294,166,167722.0
167722,Godfrey Ho,-311.939491,123,167723.0
167723,Osman F. Seden,-356.848048,130,167724.0


In [24]:
# Ranking of directors that are known also for their acting 
# (yes I did this one because I love Quentin Tarantino and he almost always play in his movies some side, silly roles)
rank_director_actors(movies_directors_df, 'primaryName', 'composite_score', good_threshold=6.0, bad_threshold=2.0)

,primaryName,custom_score,total_movies,rank
0,Quentin Tarantino,72.937783,12,1.0
1,Martin Scorsese,10.338639,36,2.0
2,Bryan Singer,6.810147,11,3.0
3,Andrew Stanton,6.086147,2,4.0
4,Mel Gibson,3.934016,5,5.0
...,...,...,...,...
33184,Orhan Elmas,-173.977429,107,33185.0
33185,Mariano Ozores,-183.960996,92,33186.0
33186,Joel Lamangan,-185.946065,101,33187.0
33187,Nejat Saydam,-189.969734,109,33188.0
